# Bayesian Experimental Design

This demo shows how to use the GPUncertaintyOptimizer class to select new training samples for a GP model. 

The samples are selected such that the predictive variance of the model is minimized.

As a toy test case, we sample the following 1-D dynamical system  
\begin{align}
y &= x \cos(x) \\
\frac{dy}{dx} &= \cos(x) - x \sin(x)
\end{align}

In [1]:
# Imports
import gpder
from gpder.gaussian_process import GaussianProcessRegressor
from gpder.gaussian_process.kernels import RegularKernel, DerivativeKernel
from gpder import GPUncertaintyOptimizer

import numpy as np 
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch, Rectangle, Arrow, FancyArrow
%matplotlib inline  

In [2]:
# Defining the function
def f(x):
    return x * np.cos(x)

# And the derivative - to be used in the derivative GP regression
def df(x):
    return np.cos(x) - x * np.sin(x)


# BED - Regular GP regression

We use the GPUncertaintyOptimizer class to select new data for the regular GP model.

In [3]:
# -- Test dataset ---------------------------------------------------------- #
X_lower, X_upper = -10, 10
X_test = np.linspace(X_lower, X_upper, 100).reshape(-1, 1)
y_test = f(X_test)
# -------------------------------------------------------------------------- #

# -- Train dataset --------------------------------------------------------- #
X_train = np.array([-2, 2, 5, 7.5]).reshape(-1, 1)
y_train = f(X_train)
# -------------------------------------------------------------------------- #

# -- Fitting the model ----------------------------------------------------- #
kernel = RegularKernel(amplitude=1.0, length_scale=1.0, noise_level=None)
print("Kernel: ")
print(kernel)
gp_reg = GaussianProcessRegressor(kernel=kernel, optimizer=None)
gp_reg.fit(X_train, y_train)
# -------------------------------------------------------------------------- #

# -- BED ------------------------------------------------------------------- #
BED_reg = GPUncertaintyOptimizer(
    gp_model=gp_reg,
    bounds={"X": (X_lower, X_upper)},
    function=f,
    random_state=42,
    verbose=True,
)
gp_reg = BED_reg.minimize_variance(
    X_util=X_test, n_iters=10, n_restarts_optimizer=10, added_noise=None
)
# -------------------------------------------------------------------------- #

Kernel: 
1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=0)
| Iter |     X      |   Target   |
----------------------------------
|  0   |   -2.00    |    0.83    |
|  0   |    2.00    |   -0.83    |
|  0   |    5.00    |    1.42    |
|  0   |    7.50    |    2.60    |
|  1   |   -6.58    |   -6.29    |
|  2   |   -4.29    |    1.75    |
|  3   |   -8.80    |    7.16    |
|  4   |    0.00    |    0.00    |
|  5   |    9.20    |   -8.97    |
|  6   |    3.53    |   -3.27    |
|  7   |    6.32    |    6.31    |
|  8   |   -5.40    |   -3.41    |
|  9   |   -9.05    |    8.44    |
|  10  |   -2.62    |    2.28    |


# BED - Derivative GP regression

We can also use the GPUncertaintyOptimizer class to select new data for the derivative GP model.

In [4]:
# -- Test dataset ---------------------------------------------------------- #
dy_test = df(X_test)
# -------------------------------------------------------------------------- #

# -- Train dataset --------------------------------------------------------- #
dy_train = df(X_train)
# -------------------------------------------------------------------------- #

# -- Fitting the derivative model ------------------------------------------ #
kernel = DerivativeKernel(
    amplitude=1.0, length_scale=1.0, noise_level=None, noise_level_der=None
)
print("Kernel: ")
print(kernel)
gp_der = GaussianProcessRegressor(kernel=kernel, optimizer=None)
gp_der.fit(X_train, y_train, dX=X_train, dy=dy_train)
# -------------------------------------------------------------------------- #

# -- BED ------------------------------------------------------------------- #
BED_der = GPUncertaintyOptimizer(
    gp_model=gp_der,
    bounds={"X": (X_lower, X_upper)},
    function=f,
    der_function=df,
    random_state=42,
    verbose=True,
)
gp_der = BED_der.minimize_variance(
    X_util=X_test, n_iters=10, n_restarts_optimizer=10, added_noise=None
)
# -------------------------------------------------------------------------- #

Kernel: 
1**2 * DerivativeRBF(length_scale=1) + WhiteKernel(noise_level=0) + WhiteKernel_der(noise_level=0)
| Iter |     X      |   Target   |
----------------------------------
|  0   |   -2.00    |    0.83    |
|  0   |    2.00    |   -0.83    |
|  0   |    5.00    |    1.42    |
|  0   |    7.50    |    2.60    |
|  1   |   -6.73    |   -6.07    |
|  2   |   -4.36    |    1.49    |
|  3   |   -9.05    |    8.43    |
|  4   |    0.15    |    0.14    |
|  5   |    9.28    |   -9.19    |
|  6   |    4.25    |   -1.89    |
|  7   |   -6.55    |   -6.32    |
|  8   |   -2.17    |    1.21    |
|  9   |   -9.39    |    9.38    |
|  10  |    7.68    |    1.36    |
